# Classifying between Imagination and Perception

For each task, orthographic, pictorial or audio, we classify to see whether the subject is imagining or perceiving the stimulus. The task selection should be done in the batch_mill main script. An output csv file is saved with the classification results. In this example, SVM is implemented. However, there is the option to implement CSP, PCA and XGBoost which are currently commented out in the classification section.

In [3]:
import mne
import numpy as np
import pandas as pd
from mne.decoding import CSP
from sklearn.model_selection import train_test_split

from tqdm import tqdm
import numpy as np
#from scipy import integrate, stats
#import antropy as ant

from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score

#### Select parameters
Either these can be batch run from the batch-papermill script, or chosen here and run locally

In [4]:
subject = '12'
session = '1'
task = 'pictorial' 
iterations = 50 

### Here the task selection is used to set other parameters
The duration is the length of the perception trial. The tag is used to identify epochs based on their event_ids.

In [5]:
if task == 'pictorial':
    tag = 'p'
    duration = 3
elif task == 'orthographic':
    print("orthographic decoding task")
    tag='t'
    duration = 3
elif task == 'audio':
    tag='s'
    duration = 2

### Loading up the data
To make the perception and imagination trials equal, we crop the imagination epochs to be of equal length to the perception duration. This is 2 seconds for audio and 3 seconds for the two visual modalities.

In [6]:
# load up files for one subject for one task of img vs. perc
# imagine_pictorial vs. perception_pictorial


perception_path = 'X:\\CompSci\\ResearchProjects\\EJONeill\\Neuroimaging\\multisensoryeeg\\processed_eeg\\epochs\\perception_'+task+'\\'
imagine_path = 'X:\\CompSci\\ResearchProjects\\EJONeill\\Neuroimaging\\multisensoryeeg\\processed_eeg\\epochs\\imagine_'+task+'\\'
datapoint = subject+'_'+session+ '_epo.fif'

perception_epochs = mne.read_epochs(perception_path + datapoint)
perception_events = mne.read_events(perception_path+ datapoint)
perception_epochs = perception_epochs.crop(tmin=0, tmax=duration)

imagination_epochs = mne.read_epochs(imagine_path + datapoint)
imagination_events = mne.read_events(imagine_path+ datapoint)
imagination_epochs = imagination_epochs.crop(tmin=0, tmax=duration)
epochs = mne.concatenate_epochs([perception_epochs,imagination_epochs])


Reading X:\CompSci\ResearchProjects\EJONeill\Neuroimaging\multisensoryeeg\processed_eeg\epochs\perception_pictorial\12_1_epo.fif ...
    Found the data of interest:
        t =       0.00 ...    3000.00 ms
        0 CTF compensation matrices available
Not setting metadata
109 matching events found
No baseline correction applied
0 projection items activated


C:\Users\hlw69\AppData\Local\Temp\ipykernel_15448\1644791318.py:11: RuntimeWarning: This filename (X:\CompSci\ResearchProjects\EJONeill\Neuroimaging\multisensoryeeg\processed_eeg\epochs\perception_pictorial\12_1_epo.fif) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst, _eve.txt or -annot.fif
  perception_events = mne.read_events(perception_path+ datapoint)


Reading X:\CompSci\ResearchProjects\EJONeill\Neuroimaging\multisensoryeeg\processed_eeg\epochs\imagine_pictorial\12_1_epo.fif ...
    Found the data of interest:
        t =       0.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
111 matching events found
No baseline correction applied
0 projection items activated


C:\Users\hlw69\AppData\Local\Temp\ipykernel_15448\1644791318.py:15: RuntimeWarning: This filename (X:\CompSci\ResearchProjects\EJONeill\Neuroimaging\multisensoryeeg\processed_eeg\epochs\imagine_pictorial\12_1_epo.fif) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst, _eve.txt or -annot.fif
  imagination_events = mne.read_events(imagine_path+ datapoint)
C:\Users\hlw69\AppData\Local\Temp\ipykernel_15448\1644791318.py:17: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs([perception_epochs,imagination_epochs])


Not setting metadata
220 matching events found
No baseline correction applied


### Dividing Epochs into Perception and Classification
We use the combine_event_ids to merge together all the perception epochs (for the three semantic categories), then do the same again for imagination. These two groups are renamed to be 'perception' and 'imagination'.

In [7]:
# Here we check that the epoch event ids are as expected
print(epochs.event_id)

{'perc_flower_p': 312, 'perc_penguin_p': 313, 'perc_guitar_p': 314, 'imag_flower_p': 303, 'imag_penguin_p': 304, 'imag_guitar_p': 305}


In [8]:

epochs = mne.epochs.combine_event_ids(epochs, old_event_ids = ['perc_penguin_'+tag,'perc_flower_'+tag,'perc_guitar_'+tag],new_event_id = {'perception':1})
epochs = mne.epochs.combine_event_ids(epochs, old_event_ids = ['imag_penguin_'+tag,'imag_flower_'+tag,'imag_guitar_'+tag], new_event_id = {'imagination':0})
print(epochs)


<EpochsArray |  220 events (all good), 0 – 3 s, baseline off, ~639.8 MB, data loaded,
 'perception': 109
 'imagination': 111>


### We create the labels for classification
#### 1 = perception
#### 0 = imagination

In [9]:
labels = epochs.events[:, -1]
print("Labels", labels)
print("Shape of epoch data ", epochs.get_data().shape)


Labels [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Shape of epoch data  (220, 124, 3073)


## Classification
Here, stratified cross validation is implemented with 5 fold. 50 iterations are set. However, the amount of iterations can be edited in the parameter settings.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
#from xgboost import XGBClassifier
from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.pipeline import make_pipeline
from mne.decoding import CSP, Scaler, Vectorizer, cross_val_multiscore,PSDEstimator,UnsupervisedSpatialFilter
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.metrics import plot_confusion_matrix, accuracy_score
import matplotlib.pyplot as plt



data = epochs.get_data()
clf = make_pipeline(
    Scaler(epochs.info),
  #  psd,
 #   UnsupervisedSpatialFilter(PCA(124), average=False),  # this has to be done due to this error: https://github.com/mne-tools/mne-python/issues/9094
#    csp,
    Vectorizer(),
    svm.SVC()
  #  LogisticRegression(solver='liblinear')  # liblinear is faster than lbfgs
  #AdaBoostClassifier(n_estimators=100)
)



accuracies = []
from random import randint # ensure that different random_state each time
for i in range(iterations):
    scores = cross_val_score(clf, data, labels, cv=5)
    print(f"Classifier = {scores}. Mean accuracy = {scores.mean()}")
    accuracies.append(scores)


Classifier = [0.75       0.68181818 0.65909091 0.77272727 0.68181818]. Mean accuracy = 0.709090909090909
Classifier = [0.75       0.68181818 0.65909091 0.77272727 0.68181818]. Mean accuracy = 0.709090909090909
Classifier = [0.75       0.68181818 0.65909091 0.77272727 0.68181818]. Mean accuracy = 0.709090909090909
Classifier = [0.75       0.68181818 0.65909091 0.77272727 0.68181818]. Mean accuracy = 0.709090909090909
Classifier = [0.75       0.68181818 0.65909091 0.77272727 0.68181818]. Mean accuracy = 0.709090909090909
